<a href="https://colab.research.google.com/github/smitaforward/Data_Science/blob/master/Text_Classification_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://realpython.com/python-keras-text-classification/

In [4]:
# load the file from drive

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


# Prep Data

In [0]:
import os
os.listdir()

['.config',
 'imdb_labelled.txt',
 'amazon_cells_labelled.txt',
 'yelp_labelled.txt',
 '.ipynb_checkpoints',
 'readme.txt',
 'sample_data']

In [0]:
import pandas as pd

filepath_dict = {'yelp':'yelp_labelled.txt',
               'amazon':'amazon_cells_labelled.txt',
               'imdb': 'imdb_labelled.txt'}

In [2]:
df_list = []
for source, filepath in filepath_dict.items():
  df = pd.read_csv(filepath, names = ['sentence','label'], sep = '\t')
  df['source'] = source
  df_list.append(df)

FileNotFoundError: ignored

In [0]:
df = pd.concat(df_list)

In [0]:
df.head()

,sentence,label,source
0,Wow... Loved this place.,1,yelp
1,Crust is not good.,0,yelp
2,Not tasty and the texture was just nasty.,0,yelp
3,Stopped by during the late May bank holiday of...,1,yelp
4,The selection on the menu was great and so wer...,1,yelp


In [0]:
df.tail()

,sentence,label,source
743,I just got bored watching Jessice Lange take h...,0,imdb
744,"Unfortunately, any virtue in this film's produ...",0,imdb
745,"In a word, it is embarrassing.",0,imdb
746,Exceptionally bad!,0,imdb
747,All in all its an insult to one's intelligence...,0,imdb


In [0]:
df.iloc[0]

sentence    Wow... Loved this place.
label                              1
source                          yelp
Name: 0, dtype: object

# Base Model

In [0]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression


In [0]:
for source in df['source'].unique():
  df_source = df[df['source'] == source]
  sentences = df_source['sentence'].values
  y = df_source['label'].values
  
  sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, 
                                                                      test_size = 0.25, random_state = 1234)
  vectorizer = CountVectorizer()
  vectorizer.fit(sentences_train)
  X_train = vectorizer.transform(sentences_train)
  X_test = vectorizer.transform(sentences_test)
  
  classifier = LogisticRegression()
  classifier.fit(X_train, y_train)
  
  score=classifier.score(X_test,y_test)
  print("Accuracy for {} data: {:.4f}".format(source, score))
  

Accuracy for yelp data: 0.8160
Accuracy for amazon data: 0.7960
Accuracy for imdb data: 0.7380


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


# keras

In [0]:
from keras.models import Sequential
from keras import layers

Using TensorFlow backend.


In [0]:
for source in df['source'].unique():
  df_source = df[df['source'] == source]
  sentences = df_source['sentence'].values
  y = df_source['label'].values
  
  sentences_train, sentences_test, y_train, y_test = train_test_split(sentences, y, 
                                                                      test_size = 0.25, random_state = 1234)
  vectorizer = CountVectorizer()
  vectorizer.fit(sentences_train)
  X_train = vectorizer.transform(sentences_train)
  X_test = vectorizer.transform(sentences_test)
  
  input_dim = X_train.shape[1]
  
  model = Sequential()
  model.add(layers.Dense(10, input_dim = input_dim, activation = 'relu'))
  model.add(layers.Dense(1, activation = 'sigmoid'))
  
  model.compile(loss = 'binary_crossentropy',
               optimizer = 'adam',
               metrics=['accuracy'])
  
  model.fit(X_train,y_train, epochs = 100, verbose = 0, 
            validation_split = 0.1,
           batch_size = 32)
  loss, accuracy = model.evaluate(X_test, y_test, verbose = 0)
  
  print("Accuracy for {} data: {:.4f}".format(source, accuracy))




Accuracy for yelp data: 0.8080
Accuracy for amazon data: 0.8400
Accuracy for imdb data: 0.7219
